In [1]:
import os
import random
import warnings
import numpy as np
import pandas as pd
import pickle as pkl
import multiprocessing

In [2]:
from aisdb import DBQuery
from datetime import datetime
from aisdb.track_gen import TrackGen
from aisdb.database import sqlfcn_callbacks
from tqdm.contrib.concurrent import process_map

In [3]:
warnings.filterwarnings("ignore")

In [4]:
random_seed = 845369
random.seed(random_seed)
np.random.seed(random_seed)

In [5]:
if not os.path.exists("pkl/"):
    os.makedirs("pkl/")

In [6]:
if not os.path.exists("csv/"):
    os.makedirs("csv/")

In [7]:
mpa_locations = pd.read_csv("./mpa-locations.csv", index_col=0, encoding="latin-1")
mpa_locations = mpa_locations[["Shape_Area", "LONG", "LAT"]]

In [8]:
temporal_range = [
#     (2015, datetime(year=2015, month=1, day=1), datetime(year=2015, month=12, day=31)),
#     (2016, datetime(year=2016, month=1, day=1), datetime(year=2016, month=12, day=31)),
#     (2017, datetime(year=2017, month=1, day=1), datetime(year=2017, month=12, day=31)),
#     (2020, datetime(year=2020, month=1, day=1), datetime(year=2020, month=12, day=31)),
    (2021, datetime(year=2021, month=1, day=1), datetime(year=2021, month=12, day=31)),
]

In [9]:
def boundingbox_distance(y, x, distance):
    y_func = (distance / 6371000.0) * (180.0 / np.pi)
    x_func = (distance / 6371000.0) * (180.0 / np.pi) / np.cos(y * np.pi / 180.0)
    # Calculates the involving bounding box given a centroid and the square-side size
    return {"ymin": y - y_func, "xmin": x - x_func, "ymax": y + y_func, "xmax": x + x_func}

In [10]:
def extract_data(data_input):
    rid, mpa = data_input
    for yyyy, t0, t1 in temporal_range:
        for extra_radius in [0, 1000, 2500, 5000, 7500, 10000]:
            search_radius = (np.sqrt(np.float(mpa.Shape_Area))) + extra_radius
            bb_distance = boundingbox_distance(x=np.float(mpa.LONG), y=np.float(mpa.LAT), distance=search_radius)
            vessel_list = DBQuery(start=t0, end=t1, callback=sqlfcn_callbacks.in_bbox_time, **bb_distance)
            vessel_list.check_idx()  # Check if tables exist and indexes are built
            vessel_list = np.vstack(vessel_list.gen_qry())  # Generate and run the query on AISDB
            f = open("pkl/%02d-%04d-%05d.pkl" % (rid, yyyy, extra_radius), "wb")
            pkl.dump([vessel_list, search_radius], f, pkl.HIGHEST_PROTOCOL)
            f.close()  # required when parallel processing
            columns = TrackGen([vessel_list]).gi_frame.f_locals["colnames"]
            try:
                df = pd.DataFrame(vessel_list, columns=columns)
                df = df.sort_values(by=["mmsi", "time"], axis=0)
            except:
                df = pd.DataFrame([], columns=columns)
            df.to_csv("csv/%02d-%04d-%05d.csv" % (rid, yyyy, extra_radius))

In [11]:
%%capture
process_map(extract_data, mpa_locations.iterrows(), max_workers=multiprocessing.cpu_count(), disable=True)